In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('mmALL_073120_csv.csv')

In [3]:
data.head()

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."
3,201990004,Canada,20,1990,North America,1,4,12.0,7.0,1990.0,...,NaN,accomodation,NaN,NaN,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed ...,canada s federal government has agreed to acqu...
4,201990005,Canada,20,1990,North America,1,5,14.0,8.0,1990.0,...,NaN,crowd dispersal,arrests,accomodation,NaN,NaN,NaN,NaN,1. dozens hurt in mohawk blockade protest the ...,protests were directed against the state due t...


In [4]:
data.keys()

Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes'],
      dtype='object')

In [5]:
# for ky in data.keys():
#     print(ky,data[ky][0])

## Yearly Change in Protest Behavior

In [6]:
yearlyChangeProtestBehavior = data[['country','year','protestnumber']]

In [7]:
yearlyChangeProtestBehavior = yearlyChangeProtestBehavior.groupby(['country','year']).sum().sort_values(['country','year']).reset_index()
yearlyChangeProtestBehavior['protestNumberChange'] = yearlyChangeProtestBehavior['protestnumber'].diff()

startIdx = yearlyChangeProtestBehavior.country != yearlyChangeProtestBehavior.country.shift(1)
yearlyChangeProtestBehavior.loc[startIdx, 'protestNumberChange'] = 0

In [8]:
yearlyChangeProtestBehavior
yearlyChangeProtestBehavior.to_csv('ProtestChange.csv')

,country,year,protestnumber,protestNumberChange
0,Afghanistan,1990,0,0.0
1,Afghanistan,1991,1,1.0
2,Afghanistan,1992,0,-1.0
3,Afghanistan,1993,0,0.0
4,Afghanistan,1994,0,0.0
...,...,...,...,...
4871,Zimbabwe,2016,153,147.0
4872,Zimbabwe,2017,15,-138.0
4873,Zimbabwe,2018,15,0.0
4874,Zimbabwe,2019,55,40.0


## Yearly Change in Protest Behavior

In [9]:
MM = data.copy()
MM = MM.sort_values(['country','year'])

In [10]:
start_df = MM[['startday', 'startmonth', 'startyear']].copy()
start_df.columns = ["day", "month", "year"]
start_df = pd.to_datetime(start_df)

In [11]:
end_df = MM[['endday', 'endmonth', 'endyear']].copy()
end_df.columns = ["day", "month", "year"]
end_df = pd.to_datetime(end_df)

In [12]:
endIdx = ~end_df.isnull()
startIdx = ~start_df.isnull()

validIdx = endIdx & startIdx

In [13]:
MM['start'] = start_df
MM['end'] = end_df

MM['protest_time'] = MM.end - MM.start + pd.Timedelta(days=1)

# MM = MM[validIdx]

In [14]:
# for i in range(1000):
#     print(MM['startday'][i], MM['endday'][i], "*", MM['startmonth'][i], MM['endmonth'][i], "*", MM['startyear'][i], MM['endyear'][i], "*",  MM['protest_time'][i])

In [15]:
reasons_dict = {'labor wage dispute':'labor',
 'land farm issue':'land',
 'police brutality': 'policebrutality',
 'political behavior, process': 'political',
 'price increases, tax policy': 'price',
 'removal of politician':'removal',
 'social restrictions':'social'}

reasons = {'labor','land','policebrutality','political','price','removal','social','other'}

MM = MM.replace(reasons_dict)

In [16]:
for resn in reasons:
    MM['reasons_'+resn] = 0

In [17]:
responses = {'accomodation',
 'arrests',
 'beatings',
 'crowd_dispersal',
 'ignore',
 'killings',
 'shootings',
 'other'}

MM = MM.replace({'crowd dispersal':'crowd_dispersal'})

In [18]:
for resp in responses:
    MM['responses_'+resp] = 0

In [19]:
for i in MM.index:
    otherFlag = 1
    for fld in ['protesterdemand1','protesterdemand2', 'protesterdemand3', 'protesterdemand4']:
        if pd.isnull(MM[fld][i]) or MM[fld][i]=='.':
            pass
        else:
            otherFlag = 0
            MM.loc[i,'reasons_'+MM[fld][i]] = 1
    if otherFlag:
        MM.loc[i,'reasons_other'] = MM['protest'][i]

In [20]:
for i in MM.index:
    otherFlag = 1
    for fld in ['stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4', 'stateresponse5', 'stateresponse6', 'stateresponse7']:
        if pd.isnull(MM[fld][i]) or MM[fld][i]=='.':
            pass
        else:
            otherFlag = 0
            MM.loc[i,'responses_'+MM[fld][i]] = 1
    if otherFlag:
        MM.loc[i,'responses_other'] = MM['protest'][i]

In [21]:
MM.sort_values(['country','year'])

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,reasons_price,reasons_removal,responses_accomodation,responses_arrests,responses_shootings,responses_other,responses_crowd_dispersal,responses_beatings,responses_killings,responses_ignore
13647,7001990000,Afghanistan,700,1990,Asia,0,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
13648,7001991001,Afghanistan,700,1991,Asia,1,1,28.0,5.0,1991.0,...,0,0,0,1,0,0,0,0,0,0
13649,7001992000,Afghanistan,700,1992,Asia,0,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
13650,7001993000,Afghanistan,700,1993,Asia,0,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
13651,7001994000,Afghanistan,700,1994,Asia,0,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11116,5522019007,Zimbabwe,552,2019,Africa,1,7,16.0,9.0,2019.0,...,0,0,0,0,0,0,0,0,0,1
11117,5522019008,Zimbabwe,552,2019,Africa,1,8,18.0,9.0,2019.0,...,0,0,0,0,0,0,1,0,0,0
11118,5522019009,Zimbabwe,552,2019,Africa,1,9,6.0,11.0,2019.0,...,0,0,0,0,0,0,1,0,0,0
11119,5522019010,Zimbabwe,552,2019,Africa,1,10,29.0,11.0,2019.0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
MM['protesterviolence'] = MM['protesterviolence'].fillna(int(0)).astype(np.int64)
# # Or
# MM = MM[~MM['protesterviolence'].isnull()]

MM['protest_time'] = MM['protest_time'].dt.days.fillna(int(0)).astype(np.int64)
# # Or
# MM = MM[~MM['protest_time'].isnull()]
# MM['protest_time'] = MM['protest_time'].dt.days.astype(np.int64)

MM['violent_response'] = MM['responses_beatings'] | MM['responses_killings'] | MM['responses_shootings']
MM['success'] = MM['responses_accomodation'].copy()

MM['violence_both'] = MM['violent_response'] | MM['protesterviolence']

In [23]:
MM['violent_protest_time'] = MM['protesterviolence']*MM['protest_time']

In [24]:
MM = MM.drop(['id', 'ccode', 'region', 'protestnumber', 'start', 'end',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources', 'notes'], axis=1)

In [25]:
MM.sort_values(['country','year'])

,country,year,protest,protesterviolence,protest_time,reasons_social,reasons_policebrutality,reasons_other,reasons_land,reasons_political,...,responses_shootings,responses_other,responses_crowd_dispersal,responses_beatings,responses_killings,responses_ignore,violent_response,success,violence_both,violent_protest_time
13647,Afghanistan,1990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13648,Afghanistan,1991,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13649,Afghanistan,1992,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13650,Afghanistan,1993,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13651,Afghanistan,1994,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11116,Zimbabwe,2019,1,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11117,Zimbabwe,2019,1,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
11118,Zimbabwe,2019,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
11119,Zimbabwe,2019,1,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [26]:
MM.groupby(['country','year']).sum().sort_values(['country','year'])

protest  protesterviolence  protest_time  reasons_social  \
country     year                                                             
Afghanistan 1990        0                  0             0               0   
            1991        1                  0             1               0   
            1992        0                  0             0               0   
            1993        0                  0             0               0   
            1994        0                  0             0               0   
...                   ...                ...           ...             ...   
Zimbabwe    2016       17                  7            17               0   
            2017        5                  3             6               0   
            2018        5                  1             5               0   
            2019       10                  4            11               0   
            2020        0                  0             0               0   

                  reasons_policebrutality  reasons_other  reasons_land  \
country     year                                                         
Afghanistan 1990                        0              0             0   
            1991                        1              0             0   
            1992                        0              0             0   
            1993                        0              0             0   
            1994                        0              0             0   
...                                   ...            ...           ...   
Zimbabwe    2016                        2              0             0   
            2017                        1              0             0   
            2018                        0              0             0   
            2019                        5              0             0   
            2020                        0              0             0   

                  reasons_political  reasons_labor  reasons_price  ...  \
country     year                                                   ...   
Afghanistan 1990                  0              0              0  ...   
            1991                  0              0              0  ...   
            1992                  0              0              0  ...   
            1993                  0              0              0  ...   
            1994                  0              0              0  ...   
...                             ...            ...            ...  ...   
Zimbabwe    2016                 10              1              4  ...   
            2017                  1              0              0  ...   
            2018                  4              1              1  ...   
            2019                  2              3              3  ...   
            2020                  0              0              0  ...   

                  responses_shootings  responses_other  \
country     year                                         
Afghanistan 1990                    0                0   
            1991                    0                0   
            1992                    0                0   
            1993                    0                0   
            1994                    0                0   
...                               ...              ...   
Zimbabwe    2016                    1                0   
            2017                    0                0   
            2018                    1                0   
            2019                    2                0   
            2020                    0                0   

                  responses_crowd_dispersal  responses_beatings  \
country     year                                                  
Afghanistan 1990                          0                   0   
            1991                          0                   0   
            1992                          0           

In [27]:
global_protest = MM[['year','protest_time']].copy()
global_protest.groupby(['year']).sum()

,protest_time
year,
1990,1133
1991,943
1992,763
1993,750
1994,703
1995,773
1996,1259
1997,1291
1998,1144


In [30]:
global_protest.to_csv('global_protest.csv')

In [31]:
MM.to_csv('main_data.csv')

In [28]:
 sum(MM['protesterviolence']*MM['violent_response']), sum(MM['protesterviolence']), sum(MM['violent_response'])

(1270, 4035, 1899)

In [29]:
 sum((1-MM['protesterviolence'])*MM['success'])/sum(1-MM['protesterviolence'])*100, sum(MM['protesterviolence']*MM['success'])/sum(MM['protesterviolence'])*100

(8.413424866514111, 10.508054522924411)